In [1]:
import os
import json

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.resources import Dataset

In [2]:
import getpass
TOKEN = getpass.getpass()

 ········


In [3]:
BUCKET = "dke/kgforge"

forge = KnowledgeGraphForge(
    "../use-cases/prod-forge-nexus.yml",
    bucket=BUCKET,
    token=TOKEN
)

In [8]:
id_ = "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/20fbc97a-fb26-43ff-8093-9136aab25dff"
self_ = "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/https:%2F%2Fbbp.epfl.ch%2Fnexus%2Fv1%2Fresources%2Fdke%2Fkgforge%2F_%2F20fbc97a-fb26-43ff-8093-9136aab25dff"

id_other_bucket = "https://bbp.epfl.ch/neurosciencegraph/data/9d64dc0d-07d1-4624-b409-cdc47ccda212"
self_other_bucket = "https://bbp.epfl.ch/nexus/v1/resources/bbp/atlas/_/https:%2F%2Fbbp.epfl.ch%2Fneurosciencegraph%2Fdata%2Fallen_ccfv3_spatial_reference_system"

all_ret = [
    (id_, "id", "same bucket"), 
    (self_, "self", "same bucket"), 
    (id_other_bucket, "id", "other bucket"), 
]


i = 0

for cb in [True, False]:
    for rs in [True, False]:
        for (el, type_, loc) in all_ret:

            print(
                f"Execution {i}", "-", 
                f"Value: {type_}", "-",
                f"Location: {loc}", "-", 
                f"Cross bucket: {cb}", "-", 
                f"Retrieve source: {rs}"
            )

            e = forge.retrieve(el, cross_bucket=cb, retrieve_source=rs)

            if not e:
                print("Not found")
            else:
                print("rev", e._store_metadata._rev)
            print("______________________")
            i += 1

Execution 0 - Value: id - Location: same bucket - Cross bucket: True - Retrieve source: True
rev 3
______________________
Execution 1 - Value: self - Location: same bucket - Cross bucket: True - Retrieve source: True
rev 3
______________________
Execution 2 - Value: id - Location: other bucket - Cross bucket: True - Retrieve source: True
<action> catch_http_error
<error> RetrievalError: failed to resolve 'https://bbp.epfl.ch/neurosciencegraph/data/9d64dc0d-07d1-4624-b409-cdc47ccda212' (latest) using resolvers of project 'dke/kgforge'

Not found
______________________
Execution 3 - Value: id - Location: same bucket - Cross bucket: True - Retrieve source: False
rev 3
______________________
Execution 4 - Value: self - Location: same bucket - Cross bucket: True - Retrieve source: False
rev 3
______________________
Execution 5 - Value: id - Location: other bucket - Cross bucket: True - Retrieve source: False
<action> catch_http_error
<error> RetrievalError: failed to resolve 'https://bbp.ep

Failures: 
- 2 - 
- 5 - 
- 8 -  To be expected: id from other bucket, and cross_bucket = False
- 11 - To be expected: id from other bucket, and cross_bucket = False